In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row

from mylib.model import three_body_model
from mylib.integration import forward_euler, backward_euler

output_notebook(hide_banner=True)

# Conservative System and Euler integration methods

## A three body problem reduced 

We consider a reduced three body problem consisting of the motion of a satellite in the framework of the attraction of the moon and the earth. For the purpose of the exercise, we assume that the system earth-moon is in circular rotation at constant speed  in a planar motion with the mass center of gravity located at the origin and that the mass of the satellite $\epsilon$ is small enough compared the mass of the earth $1-\mu$ and the mass of the moon $\mu$ to so that we can neglect its impact on the earth-moon system.  We also assume that the motion of the satellite is governed by the attraction of the two bodies earth and moon through the Newton gravitation law.

The motion of the satellite in the complex plane satisfies the equation: 

$$
\epsilon\, {\mathrm d}_t^2 Y = \frac{\epsilon(1-\mu)}{\vert\vert A-Y\vert\vert^2}\, \frac{A-Y}{\vert\vert A-Y\vert\vert}+ \frac{\epsilon\mu}{\vert\vert B-Y\vert\vert^2}\, \frac{B-Y}{\vert\vert B-Y\vert\vert}.
$$

In order to eliminate the factor $e^{it}$ in $A=-\mu\,e^{it}$ and  $A=(1-\mu)e^{it}$, we introduce the variable $y=e^{-it}\,Y = y_1+i\,y_2$. In this new referential the earth and the moon are motionless. We have $Y=e^{it}y$ and ${\mathrm d}_t^2 Y = -e^{it}y+2\,i\,e^{it}{\mathrm d}_t y+e^{it}{\mathrm d}_t^2 y$ and the equation of motion thus read:

$$
{\mathrm d}_t^2 y + 2\,i\,{\mathrm d}_t y-y= (1-\mu)\,\frac{-\mu-y}{\vert\vert \mu+y\vert\vert^3}+ \mu\, \frac{1-\mu-y}{\vert\vert 1-\mu-y\vert\vert^3}.
$$

Introducing the real and imaginary parts of $y$ and then switching to a first order system of differential equations, we obtain:

$$
\begin{array}{rcl} 
{\mathrm d}_t y_1 & = & y_3, \\ 
{\mathrm d}_t y_2 & = & y_4, \\ 
{\mathrm d}_t y_3 & = & y_1+2\,y_4- (1-\mu)(y_1+\mu)/r_1^3 - \mu (y_1-1+\mu)/r_2^3, \\
{\mathrm d}_t y_4 & = & y_2-2\,y_3- (1-\mu)y_2/r_1^3 - \mu y_2/r_2^3,
\end{array} 
$$

with $r_1=((y_1+\mu)^2+y_2^2)^{1/2}$ and $((y_1-1+\mu)^2+y_2^2)^{1/2}$.

For the initial values given below, the motion of the satellite is on a periodic orbit with period $T \approx 17.0652165601579625588917206249$.

In [2]:
# Initialization

yini = (-0.5655899165951338, 0.601315396569226, -0.45171183358756384, 0.23073427996775764)
#yini = (0.994, 0., 0., -2.00158510637908252240537862224)

tini = 0.
tend = 18.

mu = 0.012277471
    
tbm = three_body_model(mu)
fcn = tbm.fcn

def compute_invariant(y):
    y1 = y[0]
    y2 = y[1]
    y3 = y[2]
    y4 = y[3]
    r1 = np.sqrt((y1+mu)*(y1+mu) + y2*y2)
    r2 = np.sqrt((y1-1+mu)*(y1-1+mu) + y2*y2)
    inv = 0.5*(y3*y3 + y4*y4) - 0.5*(y1*y1 + y2*y2) - (1-mu)/r1 - mu/r2
    return inv

inv_ini = compute_invariant(yini)

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order(4)5 due to Dormand and Prince with stepsize control (RK45) with very fine tolerances.

In [8]:
def plot_quasi_exact_sol():
    
    tini = 0. 
    tend = 18
    
    tol = 1.e-12
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

    inv = compute_invariant(sol.y)

    fig_sol = figure(x_range=(-1.5,1.5), y_range=(-1.5,1.5), width=450, height=450, title="Arenstorf orbit")
    fig_sol.line(sol.y[0], sol.y[1], legend_label="Quasi exact solution", line_width=2)
    fig_sol.circle(0., 0., size=10, color="black")
    fig_sol.circle(1., 0., size=5, color="black")
    
    eps = 1.e-9
    fig_inv = figure(x_range=(tini,tend), y_range=(-eps,eps), width=450, height=450, 
                     title="Invariant - (initial invariant)")
    fig_inv.line(sol.t, inv-inv_ini, line_width=2)
    
    show(row(fig_sol, fig_inv))
    
    fig_sol_bis= figure(x_range=(tini, tend+2), width=950, height=350, title="Solution")
    fig_sol_bis.line(sol.t, sol.y[0], line_width=2, legend_label='y1')
    fig_sol_bis.line(sol.t, sol.y[1], line_width=2, color="crimson", legend_label='y2')
    fig_sol_bis.line(sol.t, sol.y[2], line_width=2, color="purple", legend_label='y3')
    fig_sol_bis.line(sol.t, sol.y[3], line_width=2, color="green", legend_label='y4')
    fig_sol_bis.legend.click_policy="hide"

    show(fig_sol_bis)
    
plot_quasi_exact_sol()

## Forward Euler

In [11]:
def plot_forward_euler():
    
    tini = 0. 
    tend = 18.
    tend_bis = 18

    tol = 1.e-12
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

    nt = 180000
    sol_fe = forward_euler(tini, tend_bis, nt, yini, fcn)
    
    inv = compute_invariant(sol.y)
    inv_fe = compute_invariant(sol_fe.y)

    fig_sol = figure(x_range=(-1.5,1.5), y_range=(-1.5,1.5), width=450, height=450, title="Arenstorf orbit")
    fig_sol.line(sol.y[0], sol.y[1], legend_label="Quasi exact solution", line_width=2)
    fig_sol.line(sol_fe.y[0], sol_fe.y[1], legend_label="Forward Euler", line_width=2, color="crimson")
    fig_sol.circle(0., 0., size=10, color="black")
    fig_sol.circle(1., 0., size=5, color="black")

    fig_inv = figure(x_range=(tini-1,tend), width=450, height=450, title="Invariant - (initial invariant)")
    fig_inv.line(sol.t, inv-inv_ini, line_width=2, legend_label="Quasi exact solution")
    fig_inv.line(sol_fe.t, inv_fe-inv_ini, line_width=2, color="crimson", legend_label="Forward Euler")
    fig_inv.legend.location = "center_right"
    
    fig_sol.legend.click_policy="hide"
    
    show(row(fig_sol, fig_inv))
    
plot_forward_euler()

## Backward Euler

In [12]:
def plot_backward_euler():
    
    tini = 0. 
    tend = 18.

    tol = 1.e-12
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

    nt = 1800
    sol_be = backward_euler(tini, tend, nt, yini, fcn)
    
    inv = compute_invariant(sol.y)
    inv_be = compute_invariant(sol_be.y)

    fig_sol = figure(x_range=(-1.5,1.5), y_range=(-1.5,1.5), width=450, height=450, title="Arenstorf orbit")
    fig_sol.line(sol.y[0], sol.y[1], legend_label="Quasi exact solution", line_width=2)
    fig_sol.line(sol_be.y[0], sol_be.y[1], legend_label="Backward Euler", line_width=2, color="green")
    fig_sol.circle(0., 0., size=10, color="black")
    fig_sol.circle(1., 0., size=5, color="black")

    
    fig_inv = figure(x_range=(tini-1,tend), width=450, height=450, title="Invariant - (initial invariant)")
    fig_inv.line(sol.t, inv-inv_ini, line_width=2, legend_label="Quasi exact solution")
    fig_inv.line(sol_be.t, inv_be-inv_ini, line_width=2, color="green", legend_label="Backward Euler")
    fig_inv.legend.location = "center_right"
    
    fig_sol.legend.click_policy="hide"

    show(row(fig_sol, fig_inv))
    
plot_backward_euler()

## RK45 on several periods

In [15]:
def plot_rk45():
    
    tini = 0. 
    tend = 3*18.
    
    tol = 1.e-12
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

    inv = compute_invariant(sol.y)

    fig_sol = figure(x_range=(-1.5,1.5), y_range=(-1.5,1.5), width=450, height=450, title="Arenstorf orbit")
    fig_sol.line(sol.y[0], sol.y[1], legend_label="RK45", line_width=2)
    
    eps = 1.e-9
    fig_inv = figure(x_range=(tini,tend), y_range=(-eps,eps), width=450, height=450, 
                     title="Invariant - (initial invariant)")
    fig_inv.line(sol.t, inv-inv_ini, line_width=2)
    fig_sol.circle(0., 0., size=10, color="black")
    fig_sol.circle(1., 0., size=5, color="black")

    
    show(row(fig_sol, fig_inv))
    
plot_rk45()